In [1]:
#imports everything needed to scrape
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import time
import random

## Opens up CSV from Stage 1

In [2]:
df = pd.read_csv ('Christies_Auction_Details.csv')

## Function to Close Cookie PopUp

In [3]:
def CookieCloser():
    try:
        cookie_event = WebDriverWait(driver, .05).until(
            EC.presence_of_element_located((By.ID ,"onetrust-accept-btn-handler"))
        )
        cookie_event.click()
    except:
        pass

## Function to Close Email PopUp

In [4]:
def ClickEmail():
    try:
        email_event = WebDriverWait(driver, .05).until(
            EC.presence_of_element_located((By.ID ,"close_signup"))
        )
        time.sleep(.05)
        email_event.click()
    except:
        pass

## Function Loads All Artworks onto the Page

In [5]:
def Click_LoadAll():
    
    load_all = driver.find_element(By.XPATH, '//*[text()="Load all"]') 
    
    desired_y = (load_all.size['height'] / 2) + load_all.location['y']
    current_y = (driver.execute_script('return window.innerHeight') / 2) + driver.execute_script('return window.pageYOffset')
    scroll_y_by = desired_y - current_y
    driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)
    
    time.sleep(1)
    load_all.click()
    print("!!!CLICK!!!")

## Function Returns You to the Top of the Page

In [6]:
def To_Top_of_Page():
    
    top_of_page = driver.find_element(By.CLASS_NAME, 'chr-header__panel')
    
    time.sleep(.2)
    driver.execute_script("arguments[0].scrollIntoView();", top_of_page)
    time.sleep(.2)

## Function Returns You to the Bottom of the Page

In [7]:
def To_Bottom_of_Page():
    
    bottom_of_page = driver.find_element(By.XPATH, '/html/body/chr-footer/footer/div/p')
    
    time.sleep(.2)
    driver.execute_script("arguments[0].scrollIntoView();", bottom_of_page)
    time.sleep(.2)

In [8]:
def Extract_Art_Work_URL(Auction_URL):
    
    Art_Work_URL = []
    
    #Online only auctions have a different layout to the regular auctions and must be navigated differently
    if "onlineonly" in Auction_URL:
        
        #Scrolls down in increments to load items on the page
        y = 1000
        for timer in range(0,20):
            driver.execute_script("window.scrollTo(0, "+str(y)+")")
            y += 1000  
            time.sleep(1)
        
        #once the scroler is done we return to the top of the page
        To_Top_of_Page()
        
        #Collects all of the artwork URLs
        main = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "main-content"))
        )
        
        ul = WebDriverWait(driver, 10,1).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="main-content"]/section/div/chr-infinite-scroll/ul'))
            )
        
        ArtWorks = ul.find_elements(By.TAG_NAME, 'li')
        
    else:
        
        #Finds and clicks load all and returns to the top of the page
        Click_LoadAll()
        To_Top_of_Page()
        
        #Collects all of the artwork URLs
        main = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "main-content"))
        )
        
        ul = WebDriverWait(driver, 10,1).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="main-content"]/section/div/ul'))
            )
        
        ArtWorks = ul.find_elements(By.TAG_NAME, 'li')
    
    for work in ArtWorks:
        try:
            URL = work.find_element(By.TAG_NAME, 'a')
            Art_Work_URL.append(URL.get_attribute('href'))
        except:
            print("error appending list of art")
        
    return Art_Work_URL
        
    

In [9]:
#given a path launches a website
Path = "/Users/ramirostorni/Documents/chromedriver"
driver = webdriver.Chrome(Path)


/var/folders/48/1t2lf7js69g6my_l4x0npdrw0000gn/T/ipykernel_28133/1808668305.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(Path)


## Data Collection Loop

In [10]:
All_Art_URL = []
for Auction_URL in df["URL"]:
    
    try:
        driver.get(Auction_URL)
        driver.maximize_window()
        CookieCloser()
#         driver.refresh()
        ClickEmail()
        List_of_URL = Extract_Art_Work_URL(Auction_URL)
        
    except:
        ClickEmail()
        print("ERROR: ", Auction_URL)
    
    All_Art_URL.append(List_of_URL)
   
        

ERROR:  https://onlineonly.christies.com/sso?SaleID=29505&SaleNumber=20823


NameError: name 'List_of_URL' is not defined

## saves all the object urls

In [ ]:
flat_list = [item for sublist in All_Art_URL for item in sublist]

In [ ]:
df = pd.DataFrame(flat_list, columns =["Art_URL"])
df.head(10)

In [ ]:
df.to_csv("Art_Object_URL.csv", index = False)